In [1]:
# install require library
!pip install openai==0.28 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


In [2]:
# Setting my Groq API Key


import openai

# with my Groq API Key
openai.api_key = "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"

# Set Groq as the API base
openai.api_base = "https://api.groq.com/openai/v1"

PlanAgent - Break Query into Subtasks

In [3]:
def plan_tasks(user_query):
    prompt = f"""Break this task into clear sub-tasks:
    "{user_query}"
    Provide the subtasks as a list.
    """
    response = openai.ChatCompletion.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    text = response['choices'][0]['message']['content']
    return [line.strip("-• ").strip() for line in text.split("\n") if line.strip()]

ToolAgent-Solve Subtasks

In [4]:
def tool_agent(task):
    prompt = f"Please solve this task:\n{task}"
    response = openai.ChatCompletion.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

Reflector-Improver Agent

In [5]:
def reflect(task, result):
    prompt = f"""
    Task: {task}
    Result: {result}
    Does this result fully and accurately answer the task? Suggest improvements if needed.
    """
    response = openai.ChatCompletion.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

Full Agentic Workflow

In [6]:
import time

def run_workflow(query):
    print("Planning Tasks...")
    tasks = plan_tasks(query)

    results = []
    for i, task in enumerate(tasks, start=1):
        print(f"\nTask {i}: {task}")

        answer = tool_agent(task)
        print("Result:", answer)

        time.sleep(3)  # Wait to stay within token limit

        improved = reflect(task, answer)
        print("Reflected Result:", improved)

        results.append({
            "task": task,
            "answer": answer,
            "improved": improved
        })

        time.sleep(3)

    return results

 Full Output

In [ ]:
final_output = run_workflow("Explain how Data Science is used in Legal Practice and give 3 examples.")

Planning Tasks...

Task 1: Here are the sub-tasks to break down the task:
Result: I'm happy to help! However, I don't have any sub-tasks to break down. You haven't provided any information. Please provide the task or sub-tasks you'd like me to assist with, and I'll do my best to help you solve them.
Reflected Result: The result partially answers the task. It acknowledges the request and asks for more information, which is a good start. However, it could be more helpful and concise. Here's a suggested improvement:

**Result:**
"I'm happy to help! However, I need more information to get started. Please provide the task or sub-tasks you'd like me to assist with, and I'll do my best to break them down and support you."

This revised response:

1. Acknowledges the request
2. Clearly states what's needed to move forward (more information)
3. Provides a brief overview of what they can do to help (break down tasks and support)

This should encourage the user to provide the necessary informatio

RateLimitError: Rate limit reached for model `llama3-70b-8192` in organization `org_01jvcjppsffbasx97w0dnnzn2b` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5738, Requested 781. Please try again in 5.183s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing